In [1]:
import pandas as pd
import tweepy
import config
import re

In [ ]:
auth = tweepy.OAuthHandler(config.CONSUMER_KEY, config.CONSUMER_SECRET)
auth.set_access_token(config.ACCESS_KEY, config.ACCESS_SECRET)

api = tweepy.API(auth)

In [138]:
colnames = ['user_id', 'created_at', 'text', 'is_retweet', 'mtkid']
twitterdata = pd.DataFrame(columns=colnames)

In [139]:
twitterdata

,user_id,created_at,text,is_retweet,mtkid


#### Function to get tweets (no retweets)

In [127]:
def get_twitter_data_norts(handle, mtkid, colnames):
    tweets = []
    for tweet in tweepy.Cursor(api.user_timeline, screen_name=handle, 
                           include_rts = False,
                           tweet_mode = 'extended',
                           wait_on_rate_limit = True,
                           wait_on_rate_limit_notify = True,
                           ).items():
        tweets.append(tweet)
    data = []
    for tweet in tweets:
        tweet_data = []
        tweet_data.append(tweet.user.id)
        tweet_data.append(tweet.created_at)
        tweet_data.append(re.sub('(&amp)|(@[^\s]+)|(#[^\s]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)','', tweet.full_text))
        try:
            tweet.retweeted_status
        except:
            tweet_data.append(False)
        else:
            tweet_data.append(True)
        tweet_data.append(mtkid)
        data.append(tweet_data)
    return pd.DataFrame(data,columns=colnames)

#### Function to get tweets (with retweets)

In [128]:
def get_twitter_data_norts(handle, mtkid, colnames):
    tweets = []
    for tweet in tweepy.Cursor(api.user_timeline, screen_name=handle, 
                           include_rts = False,
                           tweet_mode = 'extended',
                           wait_on_rate_limit = True,
                           wait_on_rate_limit_notify = True,
                           ).items():
        tweets.append(tweet)
    data = []
    for tweet in tweets:
        tweet_data = []
        tweet_data.append(tweet.user.id)
        tweet_data.append(tweet.created_at)
        tweet_data.append(re.sub('(&amp)|(RT)|(@[^\s]+)|(#[^\s]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)','', tweet.full_text))
        try:
            tweet.retweeted_status
        except:
            tweet_data.append(False)
        else:
            tweet_data.append(True)
        tweet_data.append(mtkid)
        data.append(tweet_data)
    return pd.DataFrame(data,columns=colnames)

In [ ]:
table = pd.read_csv()

### iterates over all rows in survey table, gets all tweets (no retweets) for all respondents

In [ ]:
for index, row in table.iterrows():
    twitterdata = twitterdata.append(get_twitter_data_norts(row['handle'], row['mtkid'], colnames))

### sample using trump

In [140]:
twitterdata = twitterdata.append(get_twitter_data_norts('realDonaldTrump', 0, colnames))

In [145]:
twitterdata.to_csv('sample.csv', encoding='utf-8', index=False)